In [3]:
import sys
import os

# Correct the path so that imports from src go through
sys.path.append(os.path.abspath(".."))

In [4]:
from src.data_retrieval import scrape_pfr_rosters, load_seasonal_stats, load_rosters
from src.data_cleaning import numeric_col, basic_roster_info, merge_roster_data, sorted_playoffs_next